In [2]:
import re

import pandas as pd

In [3]:
NHIS_Varriable_Summary_file = open("..//data//2016//NHIS_Varriable_Summary.txt", "r")
NHIS_Varriable_Summary = NHIS_Varriable_Summary_file.read()

NHIS_Varriable_Desc_file = open("..//data//2016//NHIS_Varriable_Desc.txt", "r")
NHIS_Varriable_Desc = NHIS_Varriable_Desc_file.read()

In [4]:
def extract_column_name(col_alias,text):
    new_text = text
    start_point = text.find(col_alias)+len(col_alias)
    #print(start_point)
    text = text[start_point:len(text)]
    #print(text)
    end_point = re.search('[0-9]{1,3}[ - ]*[0-9]{1,3} [0-9]{1}', text).span()[0]
    #print(end_point)
    col_name = new_text[start_point:start_point+end_point]
    if col_name.find("-")>=0:
        col_list = col_name.split(" ")
        col_list = col_list[0:len(col_list)-3]
        #print(col_list)
        col_name = " ".join(col_list)
    col_name=col_name.strip()
    if col_name.find(col_alias)==0:
        col_name = re.sub(col_alias," ",col_name).strip()
    return col_name

In [5]:
def extract_value_label(col_alias,text):
    new_text = text[text.find("Final Documentation Name: "+col_alias):len(text)]
    if new_text.find('Page')!=-1 and new_text.find('Page')< new_text.find('Question ID'):
        new_text=new_text[0:new_text.find('Page')]
    else:
        new_text=new_text[0:new_text.find('Question ID')]
    text_list=new_text.split("\n")
    text_list=[text for text in text_list if len(text)>=1]
    text_pd=pd.DataFrame(text_list)
    text_pd['number_start']=text_pd[0].apply(lambda x : len(re.findall('^[0-9]',x)))
    map_span = re.search('[1]{1,}$',"".join([str(text) for text in text_pd['number_start'].tolist()])).span()
    text_pd=text_pd[map_span[0]:map_span[1]]
    value_pd=pd.DataFrame(text_pd[0].apply(lambda x : x.split(" ")[0]))
    value_pd.columns=['label']
    value_pd['value']= text_pd[0].apply(lambda x : " ".join(x.split(" ")[1:len(x.split(" "))]))
    value_pd.index=range(0,len(value_pd.index))
    return value_pd

In [29]:
column_alias = "AHCAFYR3"

print(extract_column_name(column_alias,NHIS_Varriable_Summary))

extract_value_label(column_alias,NHIS_Varriable_Desc)

Couldn't afford dental care, past 12 m


,label,value
0,1,Yes
1,2,No
2,7,Refused
3,8,Not ascertained
4,9,Don't know
